In [111]:
import re
import json
import os

from typing import Union
from datetime import datetime, date

In [84]:
df = pd.read_csv("data/raw_csv_files/eu_mrv_shipping_data_2018.csv")

In [85]:
pd.set_option('display.max_columns', None)

In [86]:
df.head()

,IMO Number,Name,Ship type,Reporting Period,Technical efficiency,Port of Registry,Home Port,Ice Class,DoC issue date,DoC expiry date,Verifier Number,Verifier Name,Verifier NAB,Verifier Address,Verifier City,Verifier Accreditation number,Verifier Country,A,B,C,D,Total fuel consumption [m tonnes],Fuel consumptions assigned to On laden [m tonnes],Total CO₂ emissions [m tonnes],CO₂ emissions from all voyages between ports under a MS jurisdiction [m tonnes],CO₂ emissions from all voyages which departed from ports under a MS jurisdiction [m tonnes],CO₂ emissions from all voyages to ports under a MS jurisdiction [m tonnes],CO₂ emissions which occurred within ports under a MS jurisdiction at berth [m tonnes],CO₂ emissions assigned to Passenger transport [m tonnes],CO₂ emissions assigned to Freight transport [m tonnes],CO₂ emissions assigned to On laden [m tonnes],Annual Total time spent at sea [hours],Annual average Fuel consumption per distance [kg / n mile],Annual average Fuel consumption per transport work (mass) [g / m tonnes · n miles],Annual average Fuel consumption per transport work (volume) [g / m³ · n miles],Annual average Fuel consumption per transport work (dwt) [g / dwt carried · n miles],Annual average Fuel consumption per transport work (pax) [g / pax · n miles],Annual average Fuel consumption per transport work (freight) [g / m tonnes · n miles],Annual average CO₂ emissions per distance [kg CO₂ / n mile],Annual average CO₂ emissions per transport work (mass) [g CO₂ / m tonnes · n miles],Annual average CO₂ emissions per transport work (volume) [g CO₂ / m³ · n miles],Annual average CO₂ emissions per transport work (dwt) [g CO₂ / dwt carried · n miles],Annual average CO₂ emissions per transport work (pax) [g CO₂ / pax · n miles],Annual average CO₂ emissions per transport work (freight) [g CO₂ / m tonnes · n miles],Through ice [n miles],Total time spent at sea [hours],Total time spent at sea through ice [hours],Fuel consumption per distance on laden voyages [kg / n mile],Fuel consumption per transport work (mass) on laden voyages [g / m tonnes · n miles],Fuel consumption per transport work (volume) on laden voyages [g / m³ · n miles],Fuel consumption per transport work (dwt) on laden voyages [g / dwt carried · n miles],Fuel consumption per transport work (pax) on laden voyages [g / pax · n miles],Fuel consumption per transport work (freight) on laden voyages [g / m tonnes · n miles],CO₂ emissions per distance on laden voyages [kg CO₂ / n mile],CO₂ emissions per transport work (mass) on laden voyages [g CO₂ / m tonnes · n miles],CO₂ emissions per transport work (volume) on laden voyages [g CO₂ / m³ · n miles],CO₂ emissions per transport work (dwt) on laden voyages [g CO₂ / dwt carried · n miles],CO₂ emissions per transport work (pax) on laden voyages [g CO₂ / pax · n miles],CO₂ emissions per transport work (freight) on laden voyages [g CO₂ / m tonnes · n miles],Additional information to facilitate the understanding of the reported average operational energy efficiency indicators,Average density of the cargo transported [m tonnes / m³]
0,5383304,ASTORIA,Passenger ship,2018,Not Applicable,NaN,NaN,NaN,05/02/2019,30/06/2020,NaN,ICS Verification Services Single Member P.C.,Hellenic Accreditation System (ESYD),"55 Filonos Str.\n185 35 Piraeus, Greece",Piraeus,1101,Greece,Yes,NaN,NaN,NaN,6307.75,NaN,20080.25,16035.42,728.59,974.78,2341.47,NaN,NaN,NaN,4170.20,139.07,NaN,NaN,NaN,311.97,NaN,442.71,NaN,NaN,NaN,993.14,NaN,NaN,4170.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6417097,MARCO POLO,Passenger ship,2018,Not Applicable,NaN,NaN,IC,05/02/2019,30/06/2020,NaN,ICS Verification Services Single Member P.C.,Hellenic Accreditation System (ESYD),"55 Filonos Str.\n185 35 Piraeus, Greece",Piraeus,1101,Greece,Yes,NaN,NaN,NaN,8125.56,NaN,25689.03,18028.86,3161.92,2155.09,2343.16,NaN,NaN,NaN,4360.57,143.81,NaN,NaN,NaN,202.42,NaN,454.65,NaN,NaN,NaN,639.96,NaN,NaN,4360.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [87]:
df.applymap(type).apply(pd.value_counts).fillna(0)

,IMO Number,Name,Ship type,Reporting Period,Technical efficiency,Port of Registry,Home Port,Ice Class,DoC issue date,DoC expiry date,Verifier Number,Verifier Name,Verifier NAB,Verifier Address,Verifier City,Verifier Accreditation number,Verifier Country,A,B,C,D,Total fuel consumption [m tonnes],Fuel consumptions assigned to On laden [m tonnes],Total CO₂ emissions [m tonnes],CO₂ emissions from all voyages between ports under a MS jurisdiction [m tonnes],CO₂ emissions from all voyages which departed from ports under a MS jurisdiction [m tonnes],CO₂ emissions from all voyages to ports under a MS jurisdiction [m tonnes],CO₂ emissions which occurred within ports under a MS jurisdiction at berth [m tonnes],CO₂ emissions assigned to Passenger transport [m tonnes],CO₂ emissions assigned to Freight transport [m tonnes],CO₂ emissions assigned to On laden [m tonnes],Annual Total time spent at sea [hours],Annual average Fuel consumption per distance [kg / n mile],Annual average Fuel consumption per transport work (mass) [g / m tonnes · n miles],Annual average Fuel consumption per transport work (volume) [g / m³ · n miles],Annual average Fuel consumption per transport work (dwt) [g / dwt carried · n miles],Annual average Fuel consumption per transport work (pax) [g / pax · n miles],Annual average Fuel consumption per transport work (freight) [g / m tonnes · n miles],Annual average CO₂ emissions per distance [kg CO₂ / n mile],Annual average CO₂ emissions per transport work (mass) [g CO₂ / m tonnes · n miles],Annual average CO₂ emissions per transport work (volume) [g CO₂ / m³ · n miles],Annual average CO₂ emissions per transport work (dwt) [g CO₂ / dwt carried · n miles],Annual average CO₂ emissions per transport work (pax) [g CO₂ / pax · n miles],Annual average CO₂ emissions per transport work (freight) [g CO₂ / m tonnes · n miles],Through ice [n miles],Total time spent at sea [hours],Total time spent at sea through ice [hours],Fuel consumption per distance on laden voyages [kg / n mile],Fuel consumption per transport work (mass) on laden voyages [g / m tonnes · n miles],Fuel consumption per transport work (volume) on laden voyages [g / m³ · n miles],Fuel consumption per transport work (dwt) on laden voyages [g / dwt carried · n miles],Fuel consumption per transport work (pax) on laden voyages [g / pax · n miles],Fuel consumption per transport work (freight) on laden voyages [g / m tonnes · n miles],CO₂ emissions per distance on laden voyages [kg CO₂ / n mile],CO₂ emissions per transport work (mass) on laden voyages [g CO₂ / m tonnes · n miles],CO₂ emissions per transport work (volume) on laden voyages [g CO₂ / m³ · n miles],CO₂ emissions per transport work (dwt) on laden voyages [g CO₂ / dwt carried · n miles],CO₂ emissions per transport work (pax) on laden voyages [g CO₂ / pax · n miles],CO₂ emissions per transport work (freight) on laden voyages [g CO₂ / m tonnes · n miles],Additional information to facilitate the understanding of the reported average operational energy efficiency indicators,Average density of the cargo transported [m tonnes / m³]
<class 'int'>,12237.0,0.0,0.0,12237.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
<class 'str'>,0.0,12237.0,12237.0,0.0,10707.0,10530.0,2103.0,2019.0,12237.0,12237.0,3508.0,12237.0,12237.0,12237.0,12237.0,12237.0,12237.0,5201.0,3851.0,3855.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12237.0,11091.0,283.0,1404.0,536.0,360.0,12237.0,11091.0,283.0,1404.0,536.0,360.0,0.0,0.0,0.0,1296.0,1236.0,49.0,131.0,0.0,0.0,1296.0,1236.0,49.0,131.0,0.0,0.0,50.0,0.0
<class 'float'>,0.0,0.0,0.0,0.0,1530.0,1707.0,10134.0,10218.0,0.0,0.0,8729.0,0.0,0.0,0.0,0.0,0.0,0.0,7036.0,8386.0,8382.0,12237.0,12237.0,12237.0,12237.0,12237.0,12237.0,12237.0,12237.0,12237.0,12237.0,12237.0,12237.0,0.0,1146.0,11954.0,10833.0,11701.0,11877.0,0.

In [88]:
for col in df.columns:
    print(df[col].value_counts())

5383304    1
9550345    1
9548835    1
9549035    1
9549267    1
          ..
9333395    1
9333400    1
9333424    1
9333436    1
9848467    1
Name: IMO Number, Length: 12237, dtype: int64
MARIA                    5
ASTORIA                  4
AROSA                    3
AURELIA                  3
SKYROS                   3
                        ..
CMA CGM CHATEAU D' IF    1
CHEMBULK MINNEAPOLIS     1
BLUE ROSE                1
APHRODITE LEADER         1
ALF POLLAK               1
Name: Name, Length: 12039, dtype: int64
Bulk carrier                  3835
Oil tanker                    1907
Container ship                1808
Chemical tanker               1364
General cargo ship            1174
Vehicle carrier                448
Ro-pax ship                    376
Gas carrier                    321
Ro-ro ship                     272
LNG carrier                    213
Passenger ship                 159
Refrigerated cargo carrier     145
Other ship types               126
Container/ro-ro car

In [89]:
string_columns = [
    "IMO Number",
    "Name",
    "Ship type",
    "Reporting Period",
    "Technical efficiency",
    "Port of Registry",
    "Home Port",
    "Ice Class",
    "DoC issue date",
    "DoC expiry date",
    "Verifier Number",
    "Verifier Name",
    "Verifier NAB",
    "Verifier Address",
    "Verifier City",
    "Verifier Accreditation number",
    "Verifier Country",
    "A",
    "B",
    "C",
    "D"
]

In [90]:
float_columns = [col for col in df.columns if col not in string_columns]

In [91]:
upper_case_columns = ["Name", "Port of Registry", "Home Port", "Verifier Name", "Verifier NAB","Verifier Address", "Verifier City", "Verifier Country"]

In [92]:
date_columns = ["DoC issue date","DoC expiry date"]

In [93]:
def clean_numerical_data(value: Union[str, int, float]) -> float:
    if value == "Division by zero!":
        return np.nan
    if isinstance(value, str):
        return np.nan
    return round(float(value), 2)

In [94]:
def make_upper_case(value: Union[str, float]) -> str:
    if isinstance(value, str):
        return value.upper()
    return value
    

In [95]:
def extract_technical_efficiency(technical_efficiency: Union[str, float]) -> float:
    
    if technical_efficiency == "Not Applicable":
        return np.nan
    
    if isinstance(technical_efficiency, str):
        try:
            extracted_value = re.findall(r"\d*\.?\d+", technical_efficiency)[0]
            return round(float(extracted_value), 2)
        except IndexError:
            return np.nan
    

In [96]:
def convert_date(date: str) -> str:
    if date == "DoC not issued":
        return np.nan

    date_object = datetime.strptime(date, "%d/%m/%Y").date()
    date_string = datetime.strftime(date_object, "%Y-%m-%d")
    
    return date_string
    

In [97]:
def clean_column(column: str):
    column = re.sub(r"[^a-zA-Z]+", "_", column) 
    column = re.sub(r"_{2,}", "", column)
    column = re.sub(r"_$", "",column)
    return column.lower()

In [98]:
df[float_columns] = df[float_columns].applymap(clean_numerical_data)

In [99]:
df[upper_case_columns] = df[upper_case_columns].applymap(make_upper_case)

In [100]:
df["Technical efficiency"] = df["Technical efficiency"].apply(extract_technical_efficiency)

In [101]:
df[date_columns] = df[date_columns].applymap(convert_date)

In [102]:
df["Reporting Period"] = df["Reporting Period"].apply(int)

In [103]:
columns_clean = [clean_column(col) for col in df.columns]

In [104]:
df.columns = columns_clean

In [105]:
df.head()

,imo_number,name,ship_type,reporting_period,technical_efficiency,port_of_registry,home_port,ice_class,doc_issue_date,doc_expiry_date,verifier_number,verifier_name,verifier_nab,verifier_address,verifier_city,verifier_accreditation_number,verifier_country,a,b,c,d,total_fuel_consumption_m_tonnes,fuel_consumptions_assigned_to_on_laden_m_tonnes,total_co_emissions_m_tonnes,co_emissions_from_all_voyages_between_ports_under_a_ms_jurisdiction_m_tonnes,co_emissions_from_all_voyages_which_departed_from_ports_under_a_ms_jurisdiction_m_tonnes,co_emissions_from_all_voyages_to_ports_under_a_ms_jurisdiction_m_tonnes,co_emissions_which_occurred_within_ports_under_a_ms_jurisdiction_at_berth_m_tonnes,co_emissions_assigned_to_passenger_transport_m_tonnes,co_emissions_assigned_to_freight_transport_m_tonnes,co_emissions_assigned_to_on_laden_m_tonnes,annual_total_time_spent_at_sea_hours,annual_average_fuel_consumption_per_distance_kg_n_mile,annual_average_fuel_consumption_per_transport_work_mass_g_m_tonnes_n_miles,annual_average_fuel_consumption_per_transport_work_volume_g_m_n_miles,annual_average_fuel_consumption_per_transport_work_dwt_g_dwt_carried_n_miles,annual_average_fuel_consumption_per_transport_work_pax_g_pax_n_miles,annual_average_fuel_consumption_per_transport_work_freight_g_m_tonnes_n_miles,annual_average_co_emissions_per_distance_kg_co_n_mile,annual_average_co_emissions_per_transport_work_mass_g_co_m_tonnes_n_miles,annual_average_co_emissions_per_transport_work_volume_g_co_m_n_miles,annual_average_co_emissions_per_transport_work_dwt_g_co_dwt_carried_n_miles,annual_average_co_emissions_per_transport_work_pax_g_co_pax_n_miles,annual_average_co_emissions_per_transport_work_freight_g_co_m_tonnes_n_miles,through_ice_n_miles,total_time_spent_at_sea_hours,total_time_spent_at_sea_through_ice_hours,fuel_consumption_per_distance_on_laden_voyages_kg_n_mile,fuel_consumption_per_transport_work_mass_on_laden_voyages_g_m_tonnes_n_miles,fuel_consumption_per_transport_work_volume_on_laden_voyages_g_m_n_miles,fuel_consumption_per_transport_work_dwt_on_laden_voyages_g_dwt_carried_n_miles,fuel_consumption_per_transport_work_pax_on_laden_voyages_g_pax_n_miles,fuel_consumption_per_transport_work_freight_on_laden_voyages_g_m_tonnes_n_miles,co_emissions_per_distance_on_laden_voyages_kg_co_n_mile,co_emissions_per_transport_work_mass_on_laden_voyages_g_co_m_tonnes_n_miles,co_emissions_per_transport_work_volume_on_laden_voyages_g_co_m_n_miles,co_emissions_per_transport_work_dwt_on_laden_voyages_g_co_dwt_carried_n_miles,co_emissions_per_transport_work_pax_on_laden_voyages_g_co_pax_n_miles,co_emissions_per_transport_work_freight_on_laden_voyages_g_co_m_tonnes_n_miles,additional_information_to_facilitate_the_understanding_of_the_reported_average_operational_energy_efficiency_indicators,average_density_of_the_cargo_transported_m_tonnes_m
0,5383304,ASTORIA,Passenger ship,2018,NaN,NaN,NaN,NaN,2019-02-05,2020-06-30,NaN,ICS VERIFICATION SERVICES SINGLE MEMBER P.C.,HELLENIC ACCREDITATION SYSTEM (ESYD),"55 FILONOS STR.\n185 35 PIRAEUS, GREECE",PIRAEUS,1101,GREECE,Yes,NaN,NaN,NaN,6307.75,NaN,20080.25,16035.42,728.59,974.78,2341.47,NaN,NaN,NaN,4170.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4170.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6417097,MARCO POLO,Passenger ship,2018,NaN,NaN,NaN,IC,2019-02-05,2020-06-30,NaN,ICS VERIFICATION SERVICES SINGLE MEMBER P.C.,HELLENIC ACCREDITATION SYSTEM (ESYD),"55 FILONOS STR.\n185 35 PIRAEUS, GREECE",PIRAEUS,1101,GREECE,Yes,NaN,NaN,NaN,8125.56,NaN,25689.03,18028.86,3161.92,2155.09,2343.16,NaN,NaN,NaN,4360.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4360.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6511128,RED STAR 1,Ro-pax ship,2018,45.57,NaN,NaN,NaN,2019-04-30,2020-06-30,NaN,VERIFAVIA (UK) LTD,UKAS,20-22 WENLOCK ROAD,LONDON,4599,UNITED KINGDOM,Yes,NaN,NaN,NaN,2205.37,NaN,6941.34,380.99,2905.22,2893.35,761.75,4368.91,2572.43,NaN,2712.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

## Convert DataFrame to JSON

In [106]:
input_dataframe = df.loc[0]

In [107]:
data_dict = json.loads(input_dataframe.to_json())

In [172]:
print(data_dict)

{'imo_number': 5383304, 'name': 'ASTORIA', 'ship_type': 'Passenger ship', 'reporting_period': 2018, 'technical_efficiency': None, 'port_of_registry': None, 'home_port': None, 'ice_class': None, 'doc_issue_date': '2019-02-05', 'doc_expiry_date': '2020-06-30', 'verifier_number': None, 'verifier_name': 'ICS VERIFICATION SERVICES SINGLE MEMBER P.C.', 'verifier_nab': 'HELLENIC ACCREDITATION SYSTEM (ESYD)', 'verifier_address': '55 FILONOS STR.\n185 35 PIRAEUS, GREECE', 'verifier_city': 'PIRAEUS', 'verifier_accreditation_number': '1101', 'verifier_country': 'GREECE', 'a': 'Yes', 'b': None, 'c': None, 'd': None, 'total_fuel_consumption_m_tonnes': 6307.75, 'fuel_consumptions_assigned_to_on_laden_m_tonnes': None, 'total_co_emissions_m_tonnes': 20080.25, 'co_emissions_from_all_voyages_between_ports_under_a_ms_jurisdiction_m_tonnes': 16035.42, 'co_emissions_from_all_voyages_which_departed_from_ports_under_a_ms_jurisdiction_m_tonnes': 728.59, 'co_emissions_from_all_voyages_to_ports_under_a_ms_juris

In [109]:
json_path = os.path.join(*["tests", "resources", "vessel_clean.json"])

with open(json_path) as file:
    vessel_clean = json.load(file)


In [110]:
vessel_clean

{'imo_number': 5383304,
 'name': 'ASTORIA',
 'ship_type': 'Passenger ship',
 'reporting_period': 2018,
 'technical_efficiency': None,
 'port_of_registry': None,
 'home_port': None,
 'ice_class': None,
 'doc_issue_date': '2019-02-05',
 'doc_expiry_date': '2020-06-30',
 'verifier_details': {'verifier_number': None,
  'verifier_name': 'ICS VERIFICATION SERVICES SINGLE MEMBER P.C.',
  'verifier_accreditation_body': 'HELLENIC ACCREDITATION SYSTEM (ESYD)',
  'verifier_address': '55 FILONOS STR.\\n185 35 PIRAEUS, GREECE',
  'verifier_city': 'PIRAEUS',
  'verifier_accreditation_number': '1101',
  'verifier_country': 'GREECE'},
 'monitoring_methods': {'a': {'value': 'Yes',
   'description': 'BDN and period stock takes of fuel tanks'},
  'b': {'value': None, 'description': 'Bunker fuel tank monitoring on-board'},
  'c': {'value': None,
   'description': 'Flow meters for applicable combustion processes'},
  'd': {'value': None, 'description': 'Direct CO2 emissions measurement'}},
 'fuel_consumpti

In [160]:
csv_content = [['IMO Number', 'Name', 'Ship type', 'Reporting Period', 'Technical efficiency', 'Port of Registry', 'Home Port', 'Ice Class', 'DoC issue date', 'DoC expiry date', 'Verifier Number', 'Verifier Name', 'Verifier NAB', 'Verifier Address', 'Verifier City', 'Verifier Accreditation number', 'Verifier Country', 'A', 'B', 'C', 'D', 'Total fuel consumption [m tonnes]', 'Fuel consumptions assigned to On laden [m tonnes]', 'Total CO₂ emissions [m tonnes]', 'CO₂ emissions from all voyages between ports under a MS jurisdiction [m tonnes]', 'CO₂ emissions from all voyages which departed from ports under a MS jurisdiction [m tonnes]', 'CO₂ emissions from all voyages to ports under a MS jurisdiction [m tonnes]', 'CO₂ emissions which occurred within ports under a MS jurisdiction at berth [m tonnes]', 'CO₂ emissions assigned to Passenger transport [m tonnes]', 'CO₂ emissions assigned to Freight transport [m tonnes]', 'CO₂ emissions assigned to On laden [m tonnes]', 'Annual Total time spent at sea [hours]', 'Annual average Fuel consumption per distance [kg / n mile]', 'Annual average Fuel consumption per transport work (mass) [g / m tonnes · n miles]', 'Annual average Fuel consumption per transport work (volume) [g / m³ · n miles]', 'Annual average Fuel consumption per transport work (dwt) [g / dwt carried · n miles]', 'Annual average Fuel consumption per transport work (pax) [g / pax · n miles]', 'Annual average Fuel consumption per transport work (freight) [g / m tonnes · n miles]', 'Annual average CO₂ emissions per distance [kg CO₂ / n mile]', 'Annual average CO₂ emissions per transport work (mass) [g CO₂ / m tonnes · n miles]', 'Annual average CO₂ emissions per transport work (volume) [g CO₂ / m³ · n miles]', 'Annual average CO₂ emissions per transport work (dwt) [g CO₂ / dwt carried · n miles]', 'Annual average CO₂ emissions per transport work (pax) [g CO₂ / pax · n miles]', 'Annual average CO₂ emissions per transport work (freight) [g CO₂ / m tonnes · n miles]', 'Through ice [n miles]', 'Total time spent at sea [hours]', 'Total time spent at sea through ice [hours]', 'Fuel consumption per distance on laden voyages [kg / n mile]', 'Fuel consumption per transport work (mass) on laden voyages [g / m tonnes · n miles]', 'Fuel consumption per transport work (volume) on laden voyages [g / m³ · n miles]', 'Fuel consumption per transport work (dwt) on laden voyages [g / dwt carried · n miles]', 'Fuel consumption per transport work (pax) on laden voyages [g / pax · n miles]', 'Fuel consumption per transport work (freight) on laden voyages [g / m tonnes · n miles]', 'CO₂ emissions per distance on laden voyages [kg CO₂ / n mile]', 'CO₂ emissions per transport work (mass) on laden voyages [g CO₂ / m tonnes · n miles]', 'CO₂ emissions per transport work (volume) on laden voyages [g CO₂ / m³ · n miles]', 'CO₂ emissions per transport work (dwt) on laden voyages [g CO₂ / dwt carried · n miles]', 'CO₂ emissions per transport work (pax) on laden voyages [g CO₂ / pax · n miles]', 'CO₂ emissions per transport work (freight) on laden voyages [g CO₂ / m tonnes · n miles]', 'Additional information to facilitate the understanding of the reported average operational energy efficiency indicators', 'Average density of the cargo transported [m tonnes / m³]'], ['5383304', 'ASTORIA', 'Passenger ship', '2018', 'Not Applicable', '', '', '', '05/02/2019', '30/06/2020', '', 'ICS Verification Services Single Member P.C.', 'Hellenic Accreditation System (ESYD)', '55 Filonos Str.\n185 35 Piraeus, Greece', 'Piraeus', '1101', 'Greece', 'Yes', '', '', '', '6307.75', '', '20080.25', '16035.42', '728.59', '974.78', '2341.47', 'N/A', 'N/A', '', '4170.2', '139.07', 'N/A', 'N/A', 'N/A', '311.97', 'N/A', '442.71', 'N/A', 'N/A', 'N/A', '993.14', 'N/A', 'N/A', '4170.2', 'N/A', '', 'N/A', 'N/A', 'N/A', '', 'N/A', '', 'N/A', 'N/A', 'N/A', '', 'N/A', '', ''], ['6417097', 'MARCO POLO', 'Passenger ship', '2018', 'Not Applicable', '', '', 'IC', '05/02/2019', '30/06/2020', '', 'ICS Verification Services Single Member P.C.', 'Hellenic Accreditation System (ESYD)', '55 Filonos Str.\n185 35 Piraeus, Greece', 'Piraeus', '1101', 'Greece', 'Yes', '', '', '', '8125.56', '', '25689.03', '18028.86', '3161.92', '2155.09', '2343.16', 'N/A', 'N/A', '', '4360.57', '143.81', 'N/A', 'N/A', 'N/A', '202.42', 'N/A', '454.65', 'N/A', 'N/A', 'N/A', '639.96', 'N/A', '', '4360.57', '', '', 'N/A', 'N/A', 'N/A', '', 'N/A', '', 'N/A', 'N/A', 'N/A', '', 'N/A', '', ''], ['6511128', 'RED STAR 1', 'Ro-pax ship', '2018', 'EIV (45.57 gCO₂/t·nm)', '', '', '', '30/04/2019', '30/06/2020', '', 'VERIFAVIA (UK) LTD', 'UKAS', '20-22 Wenlock Road', 'London', '4599', 'United Kingdom', 'Yes', '', '', '', '2205.37', 'N/A', '6941.34', '380.99', '2905.22', '2893.35', '761.75', '4368.91', '2572.43', 'N/A', '2712', '54.43', 'N/A', 'N/A', 'N/A', '0.66', '0.81', '171.31', 'N/A', 'N/A', 'N/A', '2.07', '2.6', 'N/A', '2712', 'N/A', '', 'N/A', 'N/A', 'N/A', '', '', '', 'N/A', 'N/A', 'N/A', '', '', '', ''], ['6602898', 'OCEAN MAJESTY', 'Passenger ship', '2018', 'EIV (31.73 gCO₂/t·nm)', 'Madeira', '', '', '06/05/2019', '30/06/2020', '', 'Bureau Veritas Certification France', 'COFRAC', "Le Triangle de l'Arche\n9, cours du Triangle", '92937 PARIS LA DEFENSE', '4-0076', 'France', 'Yes', '', '', '', '6151.4', '', '19478.27', '14233.6', '982.5', '1374.1', '2887.16', 'N/A', 'N/A', '', '4286.21', '120.93', 'N/A', 'N/A', 'N/A', '355.05', 'N/A', '382.91', 'N/A', 'N/A', 'N/A', '1124.27', 'N/A', 'N/A', '4286.21', 'N/A', '', 'N/A', 'N/A', 'N/A', '', 'N/A', '', 'N/A', 'N/A', 'N/A', '', 'N/A', '', ''], ['6703343', 'EQUALITY', 'Other ship types', '2018', 'Not Applicable', 'Beirut', '', '', '30/06/2019', '30/06/2020', '', 'Dromon Bureau of Shipping', 'Hellenic Accreditation System', '2-4 Iakovou Dragatsi street, 18535', 'Piraeus', '1128', 'Greece', 'Yes', '', '', '', '549', '', '1724.63', '0', '938.13', '753.25', '33.25', 'N/A', 'N/A', '', '1008', '63.46', '91.14', 'N/A', 'Division by zero!', 'N/A', 'N/A', '199.36', '286.3', 'N/A', 'Division by zero!', 'N/A', 'N/A', 'N/A', '1008', 'N/A', '', '', 'N/A', '', 'N/A', 'N/A', '', '', 'N/A', '', 'N/A', 'N/A', '', ''], ['7037806', 'BOHUS', 'Ro-pax ship', '2018', 'EIV (146 gCO₂/t·nm)', '', '', 'IC', '30/04/2019', '30/06/2020', '', 'DNV GL', 'German national accreditation body (DAkkS)', 'Brooktorkai 18', '20457 Hamburg', 'D-VS-16026-01-00', 'Germany', 'Yes', '', '', '', '5225', 'N/A', '16751.35', '14623.63', '0', '0', '2127.72', '14238.65', '2512.7', 'N/A', '3753.82', '104.94', 'N/A', 'N/A', 'N/A', '317.41', '64.5', '336.43', 'N/A', 'N/A', 'N/A', '1017.63', '206.78', '0', '3753.82', '0', '', 'N/A', 'N/A', 'N/A', '', '', '', 'N/A', 'N/A', 'N/A', '', '', '', ''], ['7043843', 'TALOS', 'Ro-ro ship', '2018', 'EEDI (60.41 gCO₂/t·nm)', 'Piraeus', 'Piraeus', '', '25/07/2019', '30/06/2020', '', 'VERIFAVIA (UK) LTD', 'UKAS', '20-22 Wenlock Road', 'London', '4599', 'United Kingdom', 'Yes', '', 'Yes', '', '3784.12', '', '11959.02', '11959.02', '0', '0', '0', 'N/A', 'N/A', '', '3835.2', '76.87', '167.15', 'N/A', 'N/A', 'N/A', 'N/A', '242.95', '528.25', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '3835.2', 'N/A', '', '', 'N/A', 'N/A', 'N/A', 'N/A', '', '', 'N/A', 'N/A', 'N/A', 'N/A', '', ''], ['7108930', 'BLACK WATCH', 'Passenger ship', '2018', '', '', '', '', '14/05/2019', '30/06/2020', '', 'DNV GL', 'German national accreditation body (DAkkS)', 'Brooktorkai 18', '20457 Hamburg', 'D-VS-16026-01-00', 'Germany', '', '', '', '', '8994.7', '', '28302.42', '20465.88', '0', '4953.31', '2883.23', 'N/A', 'N/A', '', '4656', '165.28', 'N/A', 'N/A', 'N/A', '284.6', 'N/A', '520.05', 'N/A', 'N/A', 'N/A', '895.52', 'N/A', 'N/A', '4656', 'N/A', '', 'N/A', 'N/A', 'N/A', '', 'N/A', '', 'N/A', 'N/A', 'N/A', '', 'N/A', '', ''], ['7128332', 'SEA WIND', 'Ro-pax ship', '2018', 'EIV (9.29 gCO₂/t·nm)', 'Tallinn', 'Tallinn', 'IB', '29/04/2019', '30/06/2020', '', 'VERIFAVIA (UK) LTD', 'UKAS', '20-22 Wenlock Road', 'London', '4599', 'United Kingdom', 'Yes', '', '', '', '4079.71', 'N/A', '12912.35', '12243.3', '0', '0', '669', '3085.35', '9157.98', 'N/A', '3787', '83.73', 'N/A',
 'N/A', 'N/A', '592.19', '54.61', '265', 'N/A', 'N/A', 'N/A', '1873.44', '172.77', '', '3787', '', '', 'N/A', 'N/A', 'N/A', '', '', '', 'N/A', 'N/A', 'N/A', '', '', '', '']]

In [161]:
columns = csv_content[0]

In [162]:
columns_clean = [clean_column(col) for col in columns]

In [163]:
columns_clean

['imo_number',
 'name',
 'ship_type',
 'reporting_period',
 'technical_efficiency',
 'port_of_registry',
 'home_port',
 'ice_class',
 'doc_issue_date',
 'doc_expiry_date',
 'verifier_number',
 'verifier_name',
 'verifier_nab',
 'verifier_address',
 'verifier_city',
 'verifier_accreditation_number',
 'verifier_country',
 'a',
 'b',
 'c',
 'd',
 'total_fuel_consumption_m_tonnes',
 'fuel_consumptions_assigned_to_on_laden_m_tonnes',
 'total_co_emissions_m_tonnes',
 'co_emissions_from_all_voyages_between_ports_under_a_ms_jurisdiction_m_tonnes',
 'co_emissions_from_all_voyages_which_departed_from_ports_under_a_ms_jurisdiction_m_tonnes',
 'co_emissions_from_all_voyages_to_ports_under_a_ms_jurisdiction_m_tonnes',
 'co_emissions_which_occurred_within_ports_under_a_ms_jurisdiction_at_berth_m_tonnes',
 'co_emissions_assigned_to_passenger_transport_m_tonnes',
 'co_emissions_assigned_to_freight_transport_m_tonnes',
 'co_emissions_assigned_to_on_laden_m_tonnes',
 'annual_total_time_spent_at_sea_hour

In [164]:
string_columns = ['imo_number',
 'name',
 'ship_type',
 'reporting_period',
 'technical_efficiency',
 'port_of_registry',
 'home_port',
 'ice_class',
 'doc_issue_date',
 'doc_expiry_date',
 'verifier_number',
 'verifier_name',
 'verifier_nab',
 'verifier_address',
 'verifier_city',
 'verifier_accreditation_number',
 'verifier_country',
 'a',
 'b',
 'c',
 'd']

In [165]:
float_columns = [col for col in columns_clean if col not in string_columns]

In [166]:
upper_case_columns = ['name',
 'port_of_registry',
 'home_port',
 'verifier_name',
 'verifier_nab',
 'verifier_address',
 'verifier_city',
 'verifier_country']

In [167]:
date_columns = ["doc_issue_date","doc_expiry_date"]

In [168]:
def convert_csv_to_dictionaries(csv_content):
    columns = [clean_column(col) for col in csv_content[0]]
    return [dict(zip(columns, row)) for row in csv_content[1:]]


In [169]:
converted_data = convert_csv_to_dictionaries(csv_content)

In [173]:
print(converted_data[0])

{'imo_number': '5383304', 'name': 'ASTORIA', 'ship_type': 'Passenger ship', 'reporting_period': '2018', 'technical_efficiency': 'Not Applicable', 'port_of_registry': '', 'home_port': '', 'ice_class': '', 'doc_issue_date': '05/02/2019', 'doc_expiry_date': '30/06/2020', 'verifier_number': '', 'verifier_name': 'ICS Verification Services Single Member P.C.', 'verifier_nab': 'Hellenic Accreditation System (ESYD)', 'verifier_address': '55 Filonos Str.\n185 35 Piraeus, Greece', 'verifier_city': 'Piraeus', 'verifier_accreditation_number': '1101', 'verifier_country': 'Greece', 'a': 'Yes', 'b': '', 'c': '', 'd': '', 'total_fuel_consumption_m_tonnes': '6307.75', 'fuel_consumptions_assigned_to_on_laden_m_tonnes': '', 'total_co_emissions_m_tonnes': '20080.25', 'co_emissions_from_all_voyages_between_ports_under_a_ms_jurisdiction_m_tonnes': '16035.42', 'co_emissions_from_all_voyages_which_departed_from_ports_under_a_ms_jurisdiction_m_tonnes': '728.59', 'co_emissions_from_all_voyages_to_ports_under_a_